In [21]:
# generate links / json states programatically by cell type or anything, probably hack-y (bespoke) but it works
from caveclient import CAVEclient
import datetime
import pandas as pd
import numpy as np
import json

client = CAVEclient()
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)
client.materialize.version = 604
# timestamp = datetime.datetime.fromtimestamp(1685002201.254304)

In [24]:
def make_json(seg_ids, hidden_ids):
    state = {
    "layers": [
      {
        "source": "precomputed://gs://lee-lab_female-adult-nerve-cord/alignmentV4/em/rechunked",
        "type": "image",
        "blend": "default",
        "shaderControls": {},
        "name": "FANC EM"
      },
      {
        "type": "segmentation",
        "mesh": "precomputed://gs://lee-lab_female-adult-nerve-cord/meshes/FANC/FANC_neurons/meshes",
        "selectedAlpha": 0.4,
        "colorSeed": 3788587020,
      "segmentColors": {
        "648518346496059082": "#99e61e",
        "648518346501300835": "#58ff4d",
        "648518346496568956": "#ff301a",
        "648518346494774179": "#0401ad"
      },
        "segments": seg_ids,
        "hiddenSegments": hidden_ids,
        "skeletonRendering": {
          "mode2d": "lines_and_points",
          "mode3d": "lines"
        },
        "name": "published FANC neurons"
      },
      {
        "type": "segmentation",
        "mesh": "precomputed://gs://lee-lab_female-adult-nerve-cord/alignmentV4/volume_meshes/meshes",
        "objectAlpha": 0.1,
        "hideSegmentZero": False,
        "ignoreSegmentInteractions": True,
        "segmentColors": {
          "1": "#bfbfbf",
          "2": "#d343d6"
        },
        "segments": [
          "1",
          "2"
        ],
        "skeletonRendering": {
          "mode2d": "lines_and_points",
          "mode3d": "lines"
        },
        "name": "region outlines"
      }
    ],
    "navigation": {
      "pose": {
        "position": {
          "voxelSize": [
            4.300000190734863,
            4.300000190734863,
            45
          ],
          "voxelCoordinates": [
          33357.125,
          105326.2578125,
          928.6965942382812
        ]
        }
      },
      "zoomFactor": 12
    },
    "showAxisLines": False,
    "showDefaultAnnotations": False,
    "perspectiveZoom": 2873.855676661592,
    "showSlices": False,
    "gpuMemoryLimit": 4000000000,
    "systemMemoryLimit": 4000000000,
    "concurrentDownloads": 64,
    "jsonStateServer": "https://global.daf-apis.com/nglstate/api/v1/post",
    "selectedLayer": {
      "layer": "published FANC neurons",
      "visible": True
    },
      "layout": {
      "type": "3d",
      "orthographicProjection": True
      }
    }
    return state

In [25]:
# get left wing MNs from cave annotation table
wing_table = client.materialize.query_table('wing_motor_neuron_table_v0')

wing_table['side'] = [x.strip()[-1] for x in wing_table.classification_system]
wingL_table = wing_table[wing_table.side.isin(['L'])]

wing_MN_table = wingL_table[~wingL_table.cell_type.isin(['nonMN_unk3','nonMN_unk2','nonMN_unk1','nonMN_PSI','tt','ttb_u','ttc_u', 'PSn_u'])] # other efferent neurons
wing_MN_table.shape # should be 29

(29, 10)

In [7]:
# get left leg MNs from cave annotation table
leg_table = client.materialize.query_table('motor_neuron_table_v7')

leg_table['side'] = [x.strip()[-1] for x in leg_table.classification_system]
leg_MN_table = leg_table[leg_table.side.isin(['L'])]

leg_MN_table.shape # should be 69

(69, 10)

In [8]:
wingL_table[wingL_table.cell_type.isin(['tt','ttb_u','ttc_u'])]

,id,created,superceded_id,valid,classification_system,cell_type,pt_supervoxel_id,pt_root_id,pt_position,side
26,31,2022-11-30 17:07:21.609525+00:00,NaN,t,PDMN_L,tt,72343330830526265,648518346489942455,"[9072, 128072, 2373]",L
27,32,2022-11-30 17:07:21.610286+00:00,NaN,t,PDMN_L,ttb_u,72343330427863868,648518346494688675,"[8464, 128664, 1591]",L
28,33,2022-11-30 17:07:21.611046+00:00,NaN,t,PDMN_L,ttc_u,72343605305769879,648518346481319311,"[8344, 135464, 1599]",L


In [9]:
wingL_table[wingL_table.cell_type.str.startswith('DL')]

,id,created,superceded_id,valid,classification_system,cell_type,pt_supervoxel_id,pt_root_id,pt_position,side
0,1,2022-11-30 17:07:21.586041+00:00,NaN,t,PDMN_L,DLM_1,72624531264718610,648518346486629017,"[17992, 120088, 2996]",L
1,2,2022-11-30 17:07:21.586847+00:00,NaN,t,PDMN_L,DLM_2,72554162520753862,648518346515087943,"[14528, 119848, 3036]",L
2,3,2022-11-30 17:07:21.587651+00:00,NaN,t,PDMN_L,DLM_3,72413355843006275,648518346486563887,"[11016, 118512, 2103]",L
3,4,2022-11-30 17:07:21.588438+00:00,NaN,t,PDMN_L,DLM_4,72483793575108971,648518346494250631,"[13152, 120160, 2610]",L
4,5,2022-11-30 17:07:21.589212+00:00,NaN,t,PDMN_L,DLM_5,73750361378386281,648518346472269618,"[49280, 117624, 915]",L


In [26]:
# MN table with links

json_base = 'https://neuromancer-seung-import.appspot.com/?json_url=https://raw.githubusercontent.com/EllenLesser/Azevedo_Lesser_Phelps_Mark_2023/main/jsons/'
show_ids = [648518346501300835,648518346494774179,648518346496059082,648518346496568956]
hidden_ids = []
title = 'figure_two'

formatted_ids = [str(x) for x in show_ids]
state = make_json(seg_ids = formatted_ids,
                        hidden_ids = hidden_ids)
json_title = str(title)

# save a json
with open('jsons/{}.json'.format(json_title), "w") as outfile:
        json.dump(state, outfile)

table_link = json_base + json_title
hyperlink = 'HYPERLINK("{}.json";"{}")'.format(table_link, json_title)


In [13]:
hyperlink

'HYPERLINK("https://neuromancer-seung-import.appspot.com/?json_url=https://raw.githubusercontent.com/EllenLesser/Azevedo_Lesser_Phelps_Mark_2023/main/jsons/ltm_dipalpha_depressor.json";"ltm_dipalpha_depressor")'

In [60]:
table_link

'https://neuromancer-seung-import.appspot.com/?json_url=https://raw.githubusercontent.com/EllenLesser/Azevedo_Lesser_Phelps_Mark_2023/main/jsons/feti_seti'

In [164]:
wingL_table[wingL_table.cell_type.isin(['nonMN_unk3','nonMN_unk2','nonMN_unk1','nonMN_PSI','tt','ttb_u','ttc_u', 'PSn_u'])]

,id,created,superceded_id,valid,classification_system,cell_type,pt_supervoxel_id,pt_root_id,pt_position,side
21,26,2022-11-30 17:07:21.605759+00:00,NaN,t,mesoAN_L,PSn_u,72907242990797785,648518346492244419,"[26432, 156112, 2594]",L
26,31,2022-11-30 17:07:21.609525+00:00,NaN,t,PDMN_L,tt,72343330830526265,648518346489942455,"[9072, 128072, 2373]",L
27,32,2022-11-30 17:07:21.610286+00:00,NaN,t,PDMN_L,ttb_u,72343330427863868,648518346494688675,"[8464, 128664, 1591]",L
28,33,2022-11-30 17:07:21.611046+00:00,NaN,t,PDMN_L,ttc_u,72343605305769879,648518346481319311,"[8344, 135464, 1599]",L
29,34,2022-11-30 17:07:21.611784+00:00,NaN,t,PDMN_L,nonMN_PSI,74172918112005839,648518346486137513,"[62584, 128136, 2213]",L
30,35,2022-11-30 17:07:21.612814+00:00,NaN,t,PDMN_L,nonMN_unk1,72413355977321122,648518346472328139,"[10492, 118012, 2403]",L
31,36,2022-11-30 17:07:21.613611+00:00,NaN,t,PDMN_L,nonMN_unk2,73328975023585836,648518346514279390,"[38776, 142936, 3792]",L
32,37,2022-11-30 17:07:21.614381+00:00,NaN,t,ADMN_L,nonMN_unk3,72483724453108761,648518346483195582,"[13824, 116860, 1883]",L


In [133]:
leg_dict = {
    "tgpro_plpro" : [648518346491659326,648518346487756866,648518346517827432,648518346517437482],
    "sternal_anterior_rotator" : [648518346479325585,648518346487504531],
    "sternal_adductor" : [648518346486895331],

    "sternal_posterior_rotator" : [648518346478992848,648518346488854257,648518346490902957,648518346489802071],
    "pleural_remotor_abd" : [648518346479077839,648518346489076201],

    "tergotr" : [648518346504856435,648518346482347156,648518346496195084,648518346492578233],
    "sternotro" : [648518346467061998,648518346515708666],
    "tr_extensor" : [648518346502537286,648518346498089882],

    "tr_flexor" : [648518346517824616,648518346488873565,648518346479840662,648518346496812333,648518346514275550,648518346514277931,648518346487577002,648518346499918803],
    "acc_tr_flexor" : [648518346490899373,648518346489470041,648518346498498324],

    "fe_reductor" : [648518346502536518,648518346492338559,648518346496372087,648518346487743188,648518346494195703,648518346493238592],
    "ti_extensor" : [648518346493238080,648518346495797355],

    "ti_flexor" : [648518346494196727,648518346496932836,648518346493836024,648518346484809885,648518346494269831],
    "acc_ti_flexor" : [648518346482926501,648518346491041571,648518346493202674,648518346479331004,648518346496057290,648518346490901677,648518346482925989,648518346499176513,648518346489731855,648518346496059082],

    "ta_levator" : [648518346496852276],
    "ta_depressor" : [648518346486890393,648518346495467855,648518346486906595,648518346517830760], ## what about   648518346491158817?
    "retro_ta_depressor" : [648518346491158817],
    # current ids: 648518346480904925, 648518346489767549, 648518346495467855, and 648518346486890393
    "ltm2_femur" : [648518346486902499,648518346515711482],
    "ltm1_tibia" : [648518346493203442,648518346491158561],
    "ltm_dipalpha" : [648518346484620291,648518346504867443],
    "uncertain" : [648518346475434081,648518346496934372]
}

In [134]:
## loop for leg mn links

json_base = 'https://neuromancer-seung-import.appspot.com/?json_url=https://raw.githubusercontent.com/EllenLesser/Azevedo_Lesser_Phelps_Mark_2023/main/jsons/'

for module in leg_dict:
        show_ids = leg_dict[module]
        hidden_ids = leg_MN_table.pt_root_id.to_list()
        title = str(module)

        formatted_ids = [str(x) for x in show_ids]
        state = make_json(seg_ids = formatted_ids,
                                hidden_ids = hidden_ids)
        json_title = str(title)

        # save a json
        with open('jsons/{}.json'.format(json_title), "w") as outfile:
                json.dump(state, outfile)

        table_link = json_base + json_title
        hyperlink = 'HYPERLINK("{}.json";"{}")'.format(table_link, json_title)
        print(hyperlink)

HYPERLINK("https://neuromancer-seung-import.appspot.com/?json_url=https://raw.githubusercontent.com/EllenLesser/Azevedo_Lesser_Phelps_Mark_2023/main/jsons/tgpro_plpro.json";"tgpro_plpro")
HYPERLINK("https://neuromancer-seung-import.appspot.com/?json_url=https://raw.githubusercontent.com/EllenLesser/Azevedo_Lesser_Phelps_Mark_2023/main/jsons/sternal_anterior_rotator.json";"sternal_anterior_rotator")
HYPERLINK("https://neuromancer-seung-import.appspot.com/?json_url=https://raw.githubusercontent.com/EllenLesser/Azevedo_Lesser_Phelps_Mark_2023/main/jsons/sternal_adductor.json";"sternal_adductor")
HYPERLINK("https://neuromancer-seung-import.appspot.com/?json_url=https://raw.githubusercontent.com/EllenLesser/Azevedo_Lesser_Phelps_Mark_2023/main/jsons/sternal_posterior_rotator.json";"sternal_posterior_rotator")
HYPERLINK("https://neuromancer-seung-import.appspot.com/?json_url=https://raw.githubusercontent.com/EllenLesser/Azevedo_Lesser_Phelps_Mark_2023/main/jsons/pleural_remotor_abd.json";"pl

In [15]:
leg_MN_table[leg_MN_table.pt_root_id.isin([648518346521586425,648518346494055054])]

,id,created,superceded_id,valid,classification_system,cell_type,pt_supervoxel_id,pt_root_id,pt_position,side


In [166]:
648518346521586425,648518346494055054

[604, 734, 749, 751, 754, 755, 756]

In [2]:
from caveclient import CAVEclient

client = CAVEclient()
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)

In [3]:
wing_table = client.materialize.query_table('wing_motor_neuron_table_v0')


In [5]:
wing_table[wing_table.cell_type.isin(['nonMN_unk3','nonMN_unk2','nonMN_unk1','nonMN_PSI','tt','ttb_u','ttc_u', 'PSn_u'])]

,id,created,superceded_id,valid,classification_system,cell_type,pt_supervoxel_id,pt_root_id,pt_position
21,26,2022-11-30 17:07:21.605759+00:00,NaN,t,mesoAN_L,PSn_u,72907242990797785,648518346492244419,"[26432, 156112, 2594]"
26,31,2022-11-30 17:07:21.609525+00:00,NaN,t,PDMN_L,tt,72343330830526265,648518346489942455,"[9072, 128072, 2373]"
27,32,2022-11-30 17:07:21.610286+00:00,NaN,t,PDMN_L,ttb_u,72343330427863868,648518346494688675,"[8464, 128664, 1591]"
28,33,2022-11-30 17:07:21.611046+00:00,NaN,t,PDMN_L,ttc_u,72343605305769879,648518346514111257,"[8344, 135464, 1599]"
29,34,2022-11-30 17:07:21.611784+00:00,NaN,t,PDMN_L,nonMN_PSI,74172918112005839,648518346486137513,"[62584, 128136, 2213]"
30,35,2022-11-30 17:07:21.612814+00:00,NaN,t,PDMN_L,nonMN_unk1,72413355977321122,648518346472328139,"[10492, 118012, 2403]"
31,36,2022-11-30 17:07:21.613611+00:00,NaN,t,PDMN_L,nonMN_unk2,73328975023585836,648518346514279390,"[38776, 142936, 3792]"
32,37,2022-11-30 17:07:21.614381+00:00,NaN,t,ADMN_L,nonMN_unk3,72483724453108761,648518346483195582,"[13824, 116860, 1883]"
60,68,2022-11-30 17:07:21.638172+00:00,NaN,t,PDMN_R,tt,74243218338162030,648518346485803646,"[63544, 126192, 2665]"
61,69,2022-11-30 17:07:21.638911+00:00,NaN,t,PDMN_R,ttb_u,74172987099839888,648518346470483710,"[63008, 130032, 2711]"


In [6]:
client.materialize.version = 604

In [7]:
wing_table = client.materialize.query_table('wing_motor_neuron_table_v0')
wing_table[wing_table.cell_type.isin(['nonMN_unk3','nonMN_unk2','nonMN_unk1','nonMN_PSI','tt','ttb_u','ttc_u', 'PSn_u'])]


,id,created,superceded_id,valid,classification_system,cell_type,pt_supervoxel_id,pt_root_id,pt_position
21,26,2022-11-30 17:07:21.605759+00:00,NaN,t,mesoAN_L,PSn_u,72907242990797785,648518346492244419,"[26432, 156112, 2594]"
26,31,2022-11-30 17:07:21.609525+00:00,NaN,t,PDMN_L,tt,72343330830526265,648518346489942455,"[9072, 128072, 2373]"
27,32,2022-11-30 17:07:21.610286+00:00,NaN,t,PDMN_L,ttb_u,72343330427863868,648518346494688675,"[8464, 128664, 1591]"
28,33,2022-11-30 17:07:21.611046+00:00,NaN,t,PDMN_L,ttc_u,72343605305769879,648518346481319311,"[8344, 135464, 1599]"
29,34,2022-11-30 17:07:21.611784+00:00,NaN,t,PDMN_L,nonMN_PSI,74172918112005839,648518346486137513,"[62584, 128136, 2213]"
30,35,2022-11-30 17:07:21.612814+00:00,NaN,t,PDMN_L,nonMN_unk1,72413355977321122,648518346472328139,"[10492, 118012, 2403]"
31,36,2022-11-30 17:07:21.613611+00:00,NaN,t,PDMN_L,nonMN_unk2,73328975023585836,648518346514279390,"[38776, 142936, 3792]"
32,37,2022-11-30 17:07:21.614381+00:00,NaN,t,ADMN_L,nonMN_unk3,72483724453108761,648518346483195582,"[13824, 116860, 1883]"
60,68,2022-11-30 17:07:21.638172+00:00,NaN,t,PDMN_R,tt,74243218338162030,648518346485803646,"[63544, 126192, 2665]"
61,69,2022-11-30 17:07:21.638911+00:00,NaN,t,PDMN_R,ttb_u,74172987099839888,648518346470483710,"[63008, 130032, 2711]"
